In [ ]:
%matplotlib inline

import utils_ted
from utils_ted import *

## Setup

We haven't really looked into the detail of how this works yet - so this is provided for self-study for those who are interested. We'll look at it closely next week.

In [ ]:
path=get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path, encoding='utf8').read().lower()

In [ ]:
print('corpus length:', len(text))

In [ ]:
!tail {path} -n25

In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars) + 1

In [ ]:
print("total chars : %s" % vocab_size)

In [ ]:
chars.insert(0, '/n')

In [ ]:
"".join(chars[0:-4])

In [ ]:
char_indices = {c:i for i, c in enumerate(chars)}
indices_char = {i:c for i, c in enumerate(chars)}

In [ ]:
text_idxs = [char_indices[c] for c in text]

In [ ]:
print(text_idxs[:10])

In [ ]:
''.join(indices_char[idx] for idx in text_idxs[:70])

## Preprocess and create model

In [ ]:
maxlen = 40
sentences_list = []
next_chars_list = []
for i in range(0, (len(text_idxs) - maxlen + 1)):
    sentences_list.append(text_idxs[i: i+maxlen])
    next_chars_list.append(text_idxs[i+1: i+maxlen+1])

In [ ]:
print("nb sequences:", len(sentences_list))

In [ ]:
sentences = np.array(sentences_list[:-2])
next_chars = np.array(next_chars_list[:-2])

In [ ]:
print(sentences.shape, next_chars.shape)

In [ ]:
n_fac = 24

In [ ]:
from keras.layers import LSTM

[Keras 2.0 release notes](https://github.com/fchollet/keras/wiki/Keras-2.0-release-notes)

```
Recurrent layers
    output_dim -> units
    init -> kernel_initializer
    inner_init -> recurrent_initializer
    added argument bias_initializer
    W_regularizer -> kernel_regularizer
    b_regularizer -> bias_regularizer
    added arguments kernel_constraint, recurrent_constraint, bias_constraint
    dropout_W -> dropout
    dropout_U -> recurrent_dropout
    consume_less -> implementation. String values have been replaced with integers: implementation 0 (default), 1 or 2.
    LSTM only: the argument forget_bias_init has been removed. Instead there is a boolean argument unit_forget_bias, defaulting to True.
```

In [ ]:
model = Sequential([
    Embedding(vocab_size, n_fac, input_length=maxlen),
    #BatchNormalization(),
    #LSTM(512, input_dim=n_fac, return_sequences=True, 
    #     dropout=0.2, recurrent_dropout=0.2, implementation=1),
    LSTM(512, input_shape=(n_fac,), return_sequences=True, implementation=1),
    #Dropout(0.2),
    #BatchNormalization(),
    #LSTM(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, implementation=1),
    LSTM(512, return_sequences=True, implementation=1),
    #Dropout(0.2),
    #BatchNormalization(),
    TimeDistributed(Dense(vocab_size)),
    Activation('softmax')    
])

In [ ]:
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy')

## Train

In [ ]:
def print_example():
    seed_string="ethics is a basic foundation of all that"
    for i in range(200):
        x = np.array([char_indices[c] for c in seed_string[-40:]])[np.newaxis,:]
        preds = np.squeeze(model.predict(x, verbose=0))[-1]
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [ ]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, epochs=1, verbose=2)

In [ ]:
print_example()

In [ ]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, epochs=1, verbose=2)

In [ ]:
print_example()

In [ ]:
model.save_weights('../data/char_rnn1.h5')